In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

from richness_mass_calib import create_richness_mass_calib

In [2]:
#Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [3]:
# Data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
)
dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']

In [4]:
rmdata = create_richness_mass_calib(dt_halos)
ascaso = Nc.ClusterMassAscaso()
mset = Ncm.MSet()
mset.set(ascaso)

rmdata.m2lnL_val(mset)

13233701.307415523

In [5]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)
mset.param_set_all_ftype(Ncm.ParamType.FREE)

fit = Ncm.Fit.new(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#..............................................................................................................
#  Minimum found with precision: |df|/f =  1.00000e-08 and |dx| =  1.00000e-05
#  Elapsed time: 00 days, 00:00:17.6139330
#  iteration            [001014]
#  function evaluations [001016]
#  gradient evaluations [000000]
#  degrees of freedom   [454469]
#  m2lnL     =     391365.650978094 (     391365.65 )
#  Fit parameters:
#     4.08763247333783     0.538092645909241    0.931306629373459    0.674689968522022    0.0645030252979038  -0.244363116863723   
#----------------------------------------------------------------------------------
# Restarting:              yes
#  - absolute improvement: 12842335.6564374      
#  - relative improvement: 0.97042659178

In [6]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  4.45393539397061    [FREE]
#   -    mup1[01]:  0.688627943224023   [FREE]
#   -    mup2[02]:  1.23735536845047    [FREE]
#   - sigmap0[03]:  0.227197277810533   [FREE]
#   - sigmap1[04]: -0.0514034833229222  [FREE]
#   - sigmap2[05]: -0.125446104089159   [FREE]


In [7]:
fit.numdiff_m2lnL_covar()  # covariance matrix

fit.log_covar()  # Prints to the log file the names and indices of the fitted parameters, their best-fit
# values, standard deviations and correlation matrix.

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   -------------------------------------------------------------------------------------------
#    mup0[07000:00] =  4.454       +/-  0.002546    |  1           |  0.9426      | -0.04927     | -0.06678     | -0.08987     | -0.08526     |
#    mup1[07000:01] =  0.6886      +/-  0.0008377   |  0.9426      |  1           |  0.2243      | -0.0882      | -0.1066      | -0.06827     |
#    mup2[07000:02] =  1.237       +/-  0.002013    | -0.04927     |  0.2243      |  1           | -0.08921     | -0.07228     |  0.06067     |
# sigmap0[07000:03] =  0.2272      +/-  0.001676    | -0.06678     | -0.0882      | -0.08921     |  1           |  0.9335      | -0.1179      |
# sigmap1[07000:04] = -0.0514      +/-  0.0005431   | -0.08987     | -0.1066      | -0.07228     |  0.9335      |  1           |  0.1768      |
# sigmap2[070

In [8]:
gtkern = Ncm.MSetTransKernGauss.new(0)
mcmc = Ncm.FitMCMC.new(fit, gtkern, Ncm.FitRunMsgs.SIMPLE)

In [9]:
cov = fit.fstate.covar.dup()
cov.scale(2.0)
gtkern.set_cov(cov)

In [10]:
# mcmc.set_data_file("MCMC_ascaso_test.fits") #First fit
mcmc.set_data_file("MCMC_ascaso_test2.fits")  # For new fit

mcmc.start_run()
mcmc.run_lre(1000, 1.0e-3)
mcmc.end_run()
mcmc.mean_covar()

# NcmFitMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 2681365653.
# NcmFitMCMC: Running first 6000 pre-runs...
#----------------------------------------------------------------------------------
# NcmFitMCMC: Calculating [006000] Markov Chain Monte Carlo runs [Multivariate Gaussian Sampler]
# Task:NcmFitMCMC, started at: Wed Oct 25 2023, 06:04:14
# NcmMSetCatalog: Current mean:   3.2925e+05   4.4542       0.6887       1.2372       0.22725     -0.05139     -0.12563    
# NcmMSetCatalog: Current msd:    0.10855      7.851e-05    2.5752e-05   5.6979e-05   4.9911e-05   1.5648e-05   4.7381e-05 
# NcmMSetCatalog: Current sd:     3.7603       0.0027197    0.00089208   0.0019738    0.001729     0.00054206   0.0016413  
# NcmMSetCatalog: Current var:    14.14        7.3966e-06   7.958e-07    3.896e-06    2.9893e-06   2.9383e-07   2.6939e-06 
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            1          
# NcmFitMCMC:a

In [11]:
fit.log_covar()

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   -------------------------------------------------------------------------------------------
#    mup0[07000:00] =  4.454       +/-  0.00245     |  1           |  0.9406      | -0.02806     | -0.03985     | -0.059       | -0.03856     |
#    mup1[07000:01] =  0.6886      +/-  0.0008307   |  0.9406      |  1           |  0.2507      | -0.06281     | -0.08028     | -0.02086     |
#    mup2[07000:02] =  1.237       +/-  0.002008    | -0.02806     |  0.2507      |  1           | -0.0987      | -0.0954      |  0.04775     |
# sigmap0[07000:03] =  0.2272      +/-  0.001668    | -0.03985     | -0.06281     | -0.0987      |  1           |  0.9265      | -0.1368      |
# sigmap1[07000:04] = -0.0514      +/-  0.0005237   | -0.059       | -0.08028     | -0.0954      |  0.9265      |  1           |  0.1638      |
# sigmap2[070

In [12]:
# First Results (23-10-2023)
# ----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   -------------------------------------------------------------------------------------------
#    mup0[07000:00] =  4.422       +/-  0.002796    |  1           |  0.9095      | -0.08324     | -0.181       | -0.09781     | -0.03739     |
#    mup1[07000:01] =  0.6794      +/-  0.0009389   |  0.9095      |  1           |  0.2498      | -0.1644      | -0.1138      | -0.06086     |
#    mup2[07000:02] =  1.25        +/-  0.00286     | -0.08324     |  0.2498      |  1           | -0.01353     | -0.02935     | -0.02563     |
# sigmap0[07000:03] =  0.1588      +/-  0.0004555   | -0.181       | -0.1644      | -0.01353     |  1           | -0.4279      | -0.7289      |
# sigmap1[07000:04] = -0.1213      +/-  0.0002001   | -0.09781     | -0.1138      | -0.02935     | -0.4279      |  1           |  0.5469      |
# sigmap2[07000:05] = -0.1209      +/-  0.002002    | -0.03739     | -0.06086     | -0.02563     | -0.7289      |  0.5469      |  1           |
#                                                   -------------------------------------------------------------------------------------------